# Setup

## Instala dependencias e reinicia kernel

In [1]:
!pip install -q -U sentence-transformers datasets
quit()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 386.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


## Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")

from sentence_transformers import SentenceTransformer, util
from datasets import load_dataset
import numpy as np
from scipy.stats import pearsonr, spearmanr

# Carregar o modelo

In [2]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Dataset

In [3]:
dataset = load_dataset("mteb/stsbenchmark-sts", split="test")

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

# Olhando como estão os dados

## Dados de Interesse

- **Score:**
  - Valor de **0 a 5** que indica o quão semanticamente similares são as sentenças.

- **Sentence1 e Sentence2:**
  - Sentenças que desejamos comparar em termos de similaridade semântica.


In [4]:
dataset[0], dataset[100]

({'split': 'test',
  'genre': 'main-captions',
  'dataset': 'MSRvid',
  'year': '2012test',
  'sid': '0024',
  'score': 2.5,
  'sentence1': 'A girl is styling her hair.',
  'sentence2': 'A girl is brushing her hair.'},
 {'split': 'test',
  'genre': 'main-captions',
  'dataset': 'MSRvid',
  'year': '2012test',
  'sid': '0636',
  'score': 0.0,
  'sentence1': 'A cat is walking around a house.',
  'sentence2': 'A woman is peeling potato.'})

# Exemplo

## Sentenças

In [11]:
sentence_1 = dataset[0]['sentence1']
sentence_2 = dataset[0]['sentence2']
sentence_1, sentence_2

('A girl is styling her hair.', 'A girl is brushing her hair.')

## Embeddings

In [6]:
embedding_1 = model.encode(sentence_1)
embedding_2 = model.encode(sentence_2)
print(embedding_1[:5], "Tamanho:", len(embedding_1))
print(embedding_2[:5], "Tamanho:", len(embedding_2))

[-0.00121798 -0.00627851  0.0128859  -0.02056447 -0.07278568] Tamanho: 384
[ 0.01228208 -0.06205559  0.04094104 -0.03353566 -0.06635215] Tamanho: 384


In [7]:
util.cos_sim(embedding_1, embedding_2).item()

0.8052219152450562

# Tarefa:

### Cálculo de Métricas para Comparação de Embeddings

- **Comparar Embeddings das Sentenças 1 e 2:**
  - Utilize uma métrica como **similaridade de cosseno** para avaliar a proximidade entre os embeddings das duas sentenças.

- **Comparar com o Score do Dataset:**
  - Aplique uma métrica como a **correlação de Pearson** para verificar a correspondência entre a similaridade calculada e o score fornecido no dataset.

In [25]:
import pandas as pd
dic = pd.DataFrame()

vector = []

vector_scorefornecido = []

for current_index in range(len(dataset)):
  sentence_1 = dataset[current_index]['sentence1']
  sentence_2 = dataset[current_index]['sentence2']
  score_fornecido = dataset[current_index]['score']

  embedding_1 = model.encode(sentence_1)
  embedding_2 = model.encode(sentence_2)

  vector.append(util.cos_sim(embedding_1, embedding_2).item())
  vector_scorefornecido.append(score_fornecido)

dic['score'] = vector

dic['score_fornecido'] = vector_scorefornecido

In [26]:
print(dic[:5])

      score  score_fornecido
0  0.805222              2.5
1  0.788625              3.6
2  0.946493              5.0
3  0.882043              4.2
4  0.355616              1.5


In [30]:
correlacao = pearsonr(dic['score'], dic['score_fornecido'])

Interpretando
correlação:

* 0.9 para mais ou para menos indica uma correlação muito forte.

* 0.7 a 0.9 positivo ou negativo indica uma correlação forte.

* 0.5 a 0.7 positivo ou negativo indica uma correlação moderada.

* 0.3 a 0.5 positivo ou negativo indica uma correlação fraca.

* 0 a 0.3 positivo ou negativo indica uma correlação desprezível.

In [32]:
print(f'Correlação de Pearson: {correlacao.statistic}')

Correlação de Pearson: 0.827406424576101
